# Image Based Digit Classification using CNNs


## Import the required libraries

In [ ]:
from pathlib import Path
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers, activations, losses, metrics, optimizers, callbacks
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

## Read in the dataset

In [ ]:
# Load the CSV file
dataset = pd.read_csv('../input/digit-recognizer/train.csv')

# Split into features/X and labels/y by reshaping all rows to 28x28x1 matrices
X = np.array([np.array(row).reshape(28,28,1) for i,row in dataset.drop(columns=['label']).iterrows()])
y = np.array(dataset.label)
print(X.shape, y.shape)

## Visualize some samples

In [ ]:
fig, axs = plt.subplots(2, 5)
fig.set_size_inches(24,10)
axs = axs.flatten()
# Iterate over all digits and plot a random sample
for i, number in enumerate(range(0,10)):
    # Select random sample
    image = X[np.random.choice(np.where(y==number)[0])]
    axs[i].imshow(image, cmap='gray')
    axs[i].set(title=f'Number {number}')
    

## Create the model

In [ ]:
model = tf.keras.Sequential(layers=[
    layers.Input(shape=X[0].shape),
    layers.Conv2D(32, 3, padding='same', activation=activations.relu),
    layers.BatchNormalization(),
    layers.Conv2D(32, 3, padding='same', activation=activations.relu),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation=activations.relu),
    layers.BatchNormalization(),
    layers.Conv2D(64, 3, padding='same', activation=activations.relu),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, padding='same', activation=activations.relu),
    layers.BatchNormalization(),
    layers.Conv2D(128, 3, padding='same', activation=activations.relu),
    layers.Flatten(),
    layers.Dropout(0.3),
    layers.Dense(units=64, activation=activations.relu),
    layers.Dense(units=32, activation=activations.relu),
    layers.Dense(units=10, activation=activations.softmax)
])
model.compile(optimizer=optimizers.Adam(learning_rate=0.002), loss=losses.CategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])
print(model.summary())

## Prepare the data

In [ ]:
# Scale the data before splitting
X = X / 255.0
# Use train_test_split with 30% as testset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Encode the train labels
y_train_enc = tf.one_hot(y_train, depth=10)

print(f'Trainset shape >> {X_train.shape} & {y_train_enc.shape}')
print(f'Testset shape >> {X_test.shape} & {y_test.shape}')

## Train the model

In [ ]:
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(X_train, y_train_enc, validation_split=0.3, batch_size=64, epochs=50, callbacks=[early_stopping])

## Evaluate the model

In [ ]:
# Plot loss and accuracy
fig, axs = plt.subplots(ncols=2)
fig.set_size_inches(12, 6)
for i, metric in enumerate(['loss', 'accuracy']):
    axs[i].plot(history.epoch, history.history[f'{metric}'], label=f'Training {metric.title()}')
    axs[i].plot(history.epoch, history.history[f'val_{metric}'], label=f'Validation {metric.title()}')
    axs[i].legend()
    axs[i].set(title=metric.title(), xlim=(0,max(history.epoch)))

In [ ]:
# Predict the test set and create confusion matrix
y_pred = np.argmax(model.predict(X_test), axis=1)
cm = confusion_matrix(y_pred, y_test, normalize='true')

# Plot the results
fig, axs = plt.subplots()
fig.set_size_inches(12, 12)
sns.heatmap(cm, cmap='Blues', fmt='.2%', annot=True, ax=axs)
axs.set(title='Confusion Matrix of Testset')
plt.show()

## Predict the Kaggle Testset

In [ ]:
# Load the testset
kaggle_dataset = pd.read_csv('../input/digit-recognizer/test.csv')
# Transform and normalize
kaggle_X = np.array([np.array(row).reshape(28,28,1) for i,row in kaggle_dataset.iterrows()]) / 255.0

# Predict all samples and create submissions DataFrame
kaggle_pred = np.argmax(model.predict(kaggle_X), axis=1)
submissions_df = pd.DataFrame(data={'ImageId': np.arange(1, len(kaggle_X)+1), 'Label': kaggle_pred})

# Save the results as CSV file
submissions_df.to_csv('submission.csv', index=False)